In [3]:
from QSAR_package.data_split import extractData,randomSpliter
from QSAR_package.feature_preprocess import correlationSelection,RFE_ranking
from QSAR_package.data_scale import dataScale
from QSAR_package.grid_search import gridSearchPlus,gridSearchBase
from QSAR_package.model_evaluation import modeling
import pandas as pd
import numpy as np
from IPython.display import SVG
import csv
from sklearn.metrics import mean_squared_error

In [4]:
spliter =extractData()
file=r'D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\C2_337_som.csv'
acc_path = r"D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\C2_337_som_acc.csv"
spliter.ExtractTrainTestData(r"D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\tr_252.csv",
                             r"D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\te_85.csv",label_name='label')
tr_x = spliter.tr_x
tr_y = spliter.tr_y
te_y = spliter.te_y

corr = correlationSelection()
corr.PearsonXX(tr_x, tr_y)
randx='som'
scaler = dataScale(scale_range=(0.1, 0.9))

tr_scaled_x = scaler.FitTransform(corr.selected_tr_x.iloc[:,:])
te_scaled_x = scaler.Transform(spliter.te_x,DataSet='test')

for rankN in ['SVR','RFR']: 
    if rankN == 'Person':
        tr_ranked_x = tr_scaled_x
        te_ranked_x = te_scaled_x
        notes1 = 'Person'
    else:
        rfe = RFE_ranking(rankN,features_num=1)
        rfe.Fit(tr_scaled_x, tr_y)
        tr_ranked_x = rfe.tr_ranked_x
        te_ranked_x = te_scaled_x.loc[:,tr_ranked_x.columns]
        notes1 = rankN+'-RFE'

    for estimatorName in ['SVR','RFR']:
        grid = gridSearchPlus(grid_estimatorName=estimatorName, fold=5, repeat=2,scoreThreshold=-0.6)
        grid.FitWithFeaturesNum(tr_ranked_x, tr_y,features_range=(8,41))   # 用RFE排序
        # grid.FitWithFeaturesNum(tr_scaled_x, tr_y,features_range=(5,23))      # 用pearson相关性排序
        
        model = modeling(grid.best_estimator,params=grid.best_params)
        model.Fit(tr_scaled_x.loc[:,grid.best_features], tr_y)
        model.Predict(te_scaled_x.loc[:,grid.best_features],te_y)
        model.CrossVal(cv='LOO')
        tr_pred = model.tr_pred_y
        te_pred = model.te_pred_y
        pred_results = pd.DataFrame([tr_y.values,tr_pred,te_y.values,te_pred],
                                    index=['tr_y','tr_pred','te_y','te_pred']).T
        path = r"D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\C2_RDK_pred_{}_{}_{}.csv".format(randx,notes1,estimatorName)
        pred_results.to_csv(path,index=False)
        with open(r"D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\C2_RDK_des_{}_{}_{}.csv".format(notes1,randx,estimatorName),'w') as fobj:
            fobj.write('\n'.join(grid.best_features))
        model.ShowResults(show_cv=True,make_fig=False)
        model.SaveResults(file[:-4]+'_results.csv',notes='{},split_seed={},gridCV=5'.format(notes1,randx),save_model=True)
        tr_pred = pred_results.tr_pred
        te_pred = pred_results.te_pred
        tr_true = pred_results.tr_y
        te_true = pred_results.te_y
        pred_results["tr_abs_error"] = abs(tr_true - tr_pred)
        pred_results["te_abs_error"] = abs(te_true - te_pred).dropna(how="all")
        tr_rmse = model.tr_evaluator.rmse
        te_rmse = model.te_evaluator.rmse
        tr_count = len(pred_results.tr_abs_error.where(pred_results.tr_abs_error < tr_rmse).dropna(how="all"))
        te_count = len(pred_results.te_abs_error.where(pred_results.te_abs_error < te_rmse).dropna(how="all"))
        tr_acc = round(tr_count/len(abs(tr_true - tr_pred)), 4)
        te_acc = round(te_count/len(abs(te_true - te_pred).dropna(how="all")), 4)
        with open(acc_path, "a+", newline='') as f:
            csv_write = csv.writer(f)
            data_row = [tr_acc, te_acc]
            csv_write.writerow(data_row)

8个特征，第1/2次gridsearch，此轮耗时00h:01m:00s
8个特征，第2/2次gridsearch，此轮耗时00h:00m:55s
9个特征，第1/2次gridsearch，此轮耗时00h:00m:54s
9个特征，第2/2次gridsearch，此轮耗时00h:00m:48s
10个特征，第1/2次gridsearch，此轮耗时00h:00m:53s
10个特征，第2/2次gridsearch，此轮耗时00h:00m:49s
11个特征，第1/2次gridsearch，此轮耗时00h:00m:52s
11个特征，第2/2次gridsearch，此轮耗时00h:00m:50s
12个特征，第1/2次gridsearch，此轮耗时00h:00m:50s
12个特征，第2/2次gridsearch，此轮耗时00h:00m:49s
13个特征，第1/2次gridsearch，此轮耗时00h:00m:46s
13个特征，第2/2次gridsearch，此轮耗时00h:00m:46s
14个特征，第1/2次gridsearch，此轮耗时00h:00m:43s
14个特征，第2/2次gridsearch，此轮耗时00h:00m:42s
15个特征，第1/2次gridsearch，此轮耗时00h:00m:42s
15个特征，第2/2次gridsearch，此轮耗时00h:00m:46s
16个特征，第1/2次gridsearch，此轮耗时00h:00m:44s
16个特征，第2/2次gridsearch，此轮耗时00h:00m:44s
17个特征，第1/2次gridsearch，此轮耗时00h:00m:45s
17个特征，第2/2次gridsearch，此轮耗时00h:00m:45s
18个特征，第1/2次gridsearch，此轮耗时00h:00m:44s
18个特征，第2/2次gridsearch，此轮耗时00h:00m:44s
19个特征，第1/2次gridsearch，此轮耗时00h:00m:43s
19个特征，第2/2次gridsearch，此轮耗时00h:00m:43s
20个特征，第1/2次gridsearch，此轮耗时00h:00m:44s
20个特征，第2/2次gridsearch，此轮耗时00h:00m:44s
21个特征，第1/2次grids

C:\Users\TYJ\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


8个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
8个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
9个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
9个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
10个特征，第1/2次gridsearch，此轮耗时00h:00m:06s
10个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
11个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
11个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
12个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
12个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
13个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
13个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
14个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
14个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
15个特征，第1/2次gridsearch，此轮耗时00h:00m:07s
15个特征，第2/2次gridsearch，此轮耗时00h:00m:07s
16个特征，第1/2次gridsearch，此轮耗时00h:00m:08s
16个特征，第2/2次gridsearch，此轮耗时00h:00m:08s
17个特征，第1/2次gridsearch，此轮耗时00h:00m:08s
17个特征，第2/2次gridsearch，此轮耗时00h:00m:08s
18个特征，第1/2次gridsearch，此轮耗时00h:00m:08s
18个特征，第2/2次gridsearch，此轮耗时00h:00m:08s
19个特征，第1/2次gridsearch，此轮耗时00h:00m:08s
19个特征，第2/2次gridsearch，此轮耗时00h:00m:08s
20个特征，第1/2次gridsearch，此轮耗时00h:00m:08s
20个特征，第2/2次gridsearch，此轮耗时00h:00m:08s
21个特征，第1/2次grids

38个特征，第2/2次gridsearch，此轮耗时00h:00m:17s
39个特征，第1/2次gridsearch，此轮耗时00h:00m:17s
39个特征，第2/2次gridsearch，此轮耗时00h:00m:17s
40个特征，第1/2次gridsearch，此轮耗时00h:00m:17s
40个特征，第2/2次gridsearch，此轮耗时00h:00m:17s
33×2次gridsearch执行完毕，总耗时00h:14m:25s，可通过best_params属性查看最优参数，通过cv_results属性查看所有结果
Training results: {'r2': 0.6465, 'rmse': 0.1321, 'mae': 0.1074}
Test results: {'r2': 0.0467, 'rmse': 0.2231, 'mae': 0.1688}
        r2    rmse     mae
tr  0.6465  0.1321  0.1074
te  0.0467  0.2231  0.1688
cv  0.0924  0.2116  0.1637
模型的结果已保存至D:\academic\01_degree_doctor\01_project\mPGES\06_QSAR\C2_337\som\C2_337_som_results.csv


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers,Sequential
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
from QSAR_package.data_split import extractData, randomSpliter
from QSAR_package.feature_preprocess import correlationSelection
from QSAR_package.data_scale import dataScale
from QSAR_package.model_evaluation import modelEvaluator
from time import time
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
from IPython.display import clear_output

In [ ]:
def Sec2Time(seconds):  # convert seconds to time
    m, s = divmod(int(seconds), 60)
    h, m = divmod(m, 60)
    return ("{:02d}h:{:02d}m:{:02d}s".format(h, m, s))

In [ ]:
class PlotLosses(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, c='b',lw=1,label="loss")
        plt.plot(self.x, self.val_losses, c='r',lw=1,label="val_loss")
        plt.legend()
        plt.show();
        
    def on_train_end(self, logs={}):
        print('Training complete')
#         plt.plot(self.x, self.losses, c='b',lw=0.8,label="training loss")
#         plt.plot(self.x, self.val_losses, c='r',lw=0.8,ls='-.',label="validation loss")
#         plt.ylabel('Loss',fontproperties='Times New Roman',fontsize=13)
#         plt.xlabel('Epoch',fontproperties='Times New Roman',fontsize=13)
#         plt.title('Model Prediction Loss (CrossEntropy)',fontproperties='Arial',
#                   fontsize=15,fontstyle='normal')
#         plt.legend()
#         plt.savefig('./cox-2/loss_fig.tif',dpi=300,bbox_inches='tight')
#         plt.show();
        
plot_losses = PlotLosses()

In [ ]:
spliter =extractData()
spliter.ExtractTrainTestData("C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/tr_929.csv",
                             "C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/te_309.csv",label_name='pValue')

tr_x = spliter.tr_x
tr_y = spliter.tr_y
te_y = spliter.te_y
corr = correlationSelection()
corr.PearsonXX(tr_x, tr_y)

randx='reset'
scaler = dataScale(scale_range=(0.1, 0.9))
tr_scaled_x = scaler.FitTransform(corr.selected_tr_x)

te_scaled_x = scaler.Transform(spliter.te_x,DataSet='test')
print('train: {}\ntest: {}'.format(len(tr_scaled_x),len(te_scaled_x)))
rs = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
rs.get_n_splits(tr_y)
tr_idxs = []
va_idxs = []
for train_index, test_index in rs.split(tr_y):
    tr_idxs.append(train_index)
    va_idxs.append(test_index)
try:
    result = pd.read_csv("C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/rdkit_result{}.csv".format(randx))
    max_r2 = result.te_r2[0]
except:
    max_r2 = 0.5
for i in range(50):
    tf.keras.backend.clear_session()
    t0 = time()
    model = Sequential()
    model.add(layers.Dense(units=50,activation='relu',name='Layer1',input_shape=(tr_scaled_x.shape[1],)))
    model.add(layers.Dense(units=50,activation='relu',name='Layer2'))
    model.add(layers.Dense(units=50,activation='relu',name='Layer3'))
    model.add(layers.Dense(units=1,activation='relu',name='OutputLayer'))


    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0005),
                 loss='mse',metrics=['mae'])
    early_stopping = EarlyStopping(monitor='val_loss', min_delta=0,patience=30, verbose=0,mode='auto')

    history = []
    val_pred = []
    val_y = []
    for i in range(len(tr_idxs)):
        tr_x_input = tr_scaled_x.iloc[tr_idxs[i],:].values
        tr_y_input = tr_y.iloc[tr_idxs[i]].values
        va_x_input = tr_scaled_x.iloc[va_idxs[i],:].values
        va_y_input = tr_y.iloc[va_idxs[i]].values
        history.append(model.fit(x=tr_x_input,y=tr_y_input,epochs=1000,validation_data=(va_x_input,va_y_input),
                            verbose=0,callbacks=[early_stopping,plot_losses]))
        val_pred.extend(model.predict(x=va_x_input).flatten())
        val_y.extend(va_y_input)
        print(Sec2Time(time()-t0))

    tr_pred = model.predict(x=tr_scaled_x.values).flatten()
    # val_pred = model.predict(x=val_scaled_x.values).flatten()
    te_pred = model.predict(x=te_scaled_x.values).flatten()

    tr_Evaluator = modelEvaluator(tr_y, tr_pred)
    val_Evaluator = modelEvaluator(val_y, val_pred)
    te_Evaluator = modelEvaluator(te_y, te_pred)

    if te_Evaluator.r2 > max_r2:
        max_r2 = te_Evaluator.r2

        print(tr_Evaluator.r2,tr_Evaluator.rmse)
        print(val_Evaluator.r2,val_Evaluator.rmse)
        print(te_Evaluator.r2,te_Evaluator.rmse)

        pred_evaluation = pd.DataFrame([randx,tr_Evaluator.r2,tr_Evaluator.rmse,val_Evaluator.r2,val_Evaluator.rmse,
                                        te_Evaluator.r2,te_Evaluator.rmse],
                                      index=['split_seed','tr_r2','tr_mse','val_r2','val_mse','te_r2','te_mse']).T
        path = "C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/rdkit_result{}.csv".format(randx)
        pred_evaluation.to_csv(path,index=False)

        pred_results = pd.DataFrame([tr_y.values,tr_pred,val_y,val_pred,te_y.values,te_pred],
                                    index=['tr_y','tr_pred','val_y','val_pred','te_y','te_pred']).T
        path =  "C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/rdkit_pred_values_{}.csv".format(randx)
        pred_results.to_csv(path,index=False)

        model.save( "C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/model_rdkit_rgr_{}.h5".format(randx))

        fig = plt.figure()
        axisMin = min(tr_y.min(),te_y.min(),tr_pred.min(),te_pred.min())-0.5
        axisMax = max(tr_y.max(),te_y.max(),tr_pred.max(),te_pred.max())+0.5
        plt.plot(tr_y,tr_pred,'xb',markersize=8)
        plt.plot(te_y,te_pred,'or',mfc='w',markersize=6)
        plt.plot([axisMin,axisMax],[axisMin,axisMax],'k',lw=1)
        plt.axis([axisMin,axisMax,axisMin,axisMax])
        plt.xlabel('pKi values (true)',fontproperties='Times New Roman',fontsize=13)
        plt.ylabel('pKi values (predict)',fontproperties='Times New Roman',fontsize=13)
        plt.legend(['training set', 'test set'], loc='best')
        plt.savefig('C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/rdkit_scatter_fig_{}.tif'.format(randx),
                    dpi=300,bbox_inches='tight')
        plt.show()

        tr_loss = []
        va_loss = []
        tr_loss_flat = []
        va_loss_flat = []
        for i in range(len(history)):
            tr_loss.append(history[i].history['loss'])
            va_loss.append(history[i].history['val_loss'])
            tr_loss_flat.extend(history[i].history['loss'])
            va_loss_flat.extend(history[i].history['val_loss'])

        step = 0
        flag = [0]
        for i in range(len(tr_loss)-1):
            step = step+len(tr_loss[i])
            flag.append(step)
        flag.append(len(tr_loss_flat))

        tr_loss_df = pd.DataFrame(tr_loss,index=['fold_{}'.format(i+1) for i in range(len(tr_loss))]).T
        val_loss_df = pd.DataFrame(va_loss,index=['fold_{}'.format(i+1) for i in range(len(va_loss))]).T

        tr_loss_df.to_csv('C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/rdkit_tr_loss_{}.csv'.format(randx),
                          index=False)
        val_loss_df.to_csv('C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/rdkit_val_loss_{}.csv'.format(randx),
                           index=False)

        h = 0.05
        x_min = 0
        x_max = len(tr_loss_flat)
        y_min = min(tr_loss_flat)-1
        y_max = max(tr_loss_flat)+1
        xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))

        Z = np.zeros_like(xx)

        for i in range(len(flag)-1):
            Z[np.where((flag[i]<xx)&(xx<=flag[i+1]))] = i+1

        fig = plt.figure(figsize=(9,6))
        plt.plot(tr_loss_flat,c='b',ls='solid',lw=0.8)
        plt.plot(va_loss_flat,c='r',ls='-.',lw=0.8)
        plt.imshow(Z, interpolation='nearest',alpha=0.5,
                   extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                   cmap=plt.cm.Greys,
                   aspect='auto', origin='lower')
        plt.xlim(-10,x_max+5)
        plt.ylim(0,2)
        for i in range(len(flag)-1):
            plt.text(np.average(flag[i:i+2]), 1, '{}th\nfold'.format(i+1), fontsize=2,color='#FF00FF',
                     fontproperties='Arial',rasterized=True,horizontalalignment='center')
        plt.ylabel('Loss',fontproperties='Times New Roman',fontsize=13)
        plt.xlabel('Epoch',fontproperties='Times New Roman',fontsize=13)
        plt.legend(['training loss', 'validation loss'], loc=(0.05,0.87))
        plt.savefig('C:/Users/zhangshd408/Desktop/recent_work/1238/reset_all/DL_results_rdkit_loss_fig_{}.tif'.format(randx),
                    dpi=300,bbox_inches='tight')
        plt.show()

In [ ]:
SVG(model_to_dot(model,show_shapes=True).create(prog='dot', format='svg'))